# Setting up a mock DB to store Songs, users, and playlists
![](https://github.com/spotify-song/DS/blob/master/references/DB_tables_illu.png?raw=true)
### Tables:
    - User:
        - id
        - display_name
    - Tokens:
        - id
        - access_token
        - token_type
        - expries_in
        - refresh_token
        - scope
        - expires_at
        - user
    - Tracks
        - id
        - danceability
        - energy
        - loudness
        - mode
        - speechiness
        - accousticness
        - instrumentalness
        - valence
        - liveness
        - tempo
        - duration_ms
        - time_signature
    - UserPlaylist
        - id
        - u_id
        - uri
        - tracks_id

## Importing db classes for tables

In [1]:
import sys
sys.path.insert(0, '../')
import api.models.my_db
from api.models.my_db import *
from api.spotify_users import UserData

In [2]:
# imports
import sys
import json
import random
import os
from os import getenv

import spotipy
import psycopg2
import numpy as np
import pandas as pd
import spotipy.util as util
from dotenv import load_dotenv
from sqlalchemy.orm import sessionmaker
from json.decoder import JSONDecodeError
from sqlalchemy import create_engine, update
from sqlalchemy.ext.declarative import declarative_base
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

load_dotenv()

client_secret = getenv('SPOTIFY_CLIENT_SECRET')
client_id = getenv('SPOTIFY_CLIENT_ID')
user_id = 'Agustinvargas'

# change for deplpoyment
uri = getenv('uri')

# Scopes: User top track; creates playlist
scope = 'playlist-modify-public user-library-read user-top-read'
cache_path = '../../token_cache/'

spot_cc = spotipy.oauth2.SpotifyOAuth(
                                username=user_id,
                                client_id=client_id,
                                client_secret=client_secret,
                                scope=scope,
                                redirect_uri=uri,
                                show_dialog=True
                                )


spot_session = spotipy.Spotify(oauth_manager=spot_cc)

## DB import flow needs to be:
    - User > tokens
    - Tracks > top_tracks
    - users/tracks > user_playlists

### Get the pertinent data using the UserData() class

In [11]:
user = UserData()

In [13]:
import pandas as pd
df = pd.DataFrame(top_50_aud_feat)

NameError: name 'top_50_aud_feat' is not defined

In [8]:
df1 = df[:30]
df2 = df[20:]

In [9]:
users_top_trx = pd.concat([df1, df2])
users_top_trx = users_top_trx.drop_duplicates(inplace=False, subset='id')

In [10]:

# drop_col = ['key', 'mode', 'instrumentalness', 'duration_ms', 'time_signature', 'analysis_url', 'track_href', 'uri', 'id', 'type']
# users_top_trx = users_top_trx.drop(drop_col, axis=1)
users_top_trx.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.830,0.159,1,-14.461,1,0.0383,0.946,0.000020,0.3620,0.189,104.950,audio_features,0akyEssGRVHstqCSWXusJL,spotify:track:0akyEssGRVHstqCSWXusJL,https://api.spotify.com/v1/tracks/0akyEssGRVHs...,https://api.spotify.com/v1/audio-analysis/0aky...,207400,4
1,0.726,0.125,5,-9.194,0,0.0803,0.835,0.000000,0.1310,0.277,92.230,audio_features,02gaYAEdeR6poHcBH1KUQF,spotify:track:02gaYAEdeR6poHcBH1KUQF,https://api.spotify.com/v1/tracks/02gaYAEdeR6p...,https://api.spotify.com/v1/audio-analysis/02ga...,183711,4
2,0.658,0.179,8,-10.866,1,0.0448,0.689,0.000000,0.1700,0.191,128.128,audio_features,0NeJjNlprGfZpeX2LQuN6c,spotify:track:0NeJjNlprGfZpeX2LQuN6c,https://api.spotify.com/v1/tracks/0NeJjNlprGfZ...,https://api.spotify.com/v1/audio-analysis/0NeJ...,238560,4
3,0.780,0.230,4,-12.706,1,0.0448,0.913,0.002790,0.0798,0.125,123.937,audio_features,6plO0gM4tUvRC9TKFGIuaN,spotify:track:6plO0gM4tUvRC9TKFGIuaN,https://api.spotify.com/v1/tracks/6plO0gM4tUvR...,https://api.spotify.com/v1/audio-analysis/6plO...,240307,4
4,0.770,0.325,7,-11.301,1,0.0322,0.899,0.000556,0.2200,0.721,103.085,audio_features,54KsfVVnN4YWI2mMrnyUcC,spotify:track:54KsfVVnN4YWI2mMrnyUcC,https://api.spotify.com/v1/tracks/54KsfVVnN4YW...,https://api.spotify.com/v1/audio-analysis/54Ks...,209652,4


In [20]:
import random

users_top_trx_id = random.sample(list(users_top_trx['id']), 5)


In [21]:
recs = spot_session.recommendations(seed_tracks=users_top_trx_id,
                                    limit=50,
                                    target_danceability=users_top_trx['danceability'].mean(),
                                    target_energy=users_top_trx[
                                                                'energy'
                                                                ].mean(),
                                    target_loudness=users_top_trx[
                                                                'loudness'
                                                                ].mean(),
                                    target_speechiness=users_top_trx[
                                                                'speechiness'
                                                                ].mean(),
                                    target_acousticness=users_top_trx[
                                                                'acousticness'
                                                                ].mean(),
                                    target_valence=users_top_trx[
                                                                'valence'
                                                                ].mean(),
                                    target_liveness=users_top_trx[
                                                                'liveness'
                                                                ].mean(),
                                    target_tempo=users_top_trx[
                                                                'tempo'
                                                                ].mean()
                            )

In [26]:
tracks = []
for _ in recs['tracks']:
    tracks.append(_['id'])
tracks

['7CyTcdRBnWs3vCCx9kPWFY',
 '20S5GpuTDgnx9nwBFp5Lpb',
 '2z3W6iMMVhRj4YYY3rBPKk',
 '6aommJFM8i3iT4inKuDSXh',
 '1cJkG6gu8P0sUfAyYHMUIb',
 '1mbuSss0OVqfS00y7yCNCZ',
 '6VT2jR1Y4KLCXKZVo1JiQU',
 '2ALEhWrcpAdOC7YYJBQIre',
 '2BWao7MNRld2ucMxswvxaU',
 '7lFcRLIrpzjPd2Ytp13Ml3',
 '6UQIDBUAknHTNyDKJnCJfe',
 '7jC9oz02D4KHAd85ej8lUe',
 '4DOPoeO6dEOMv9fbwCBKos',
 '1ZnGM9Hb83H5ySQzNK2LJ0',
 '1xjG9J2yC5kCWJfwqYvnk9',
 '6lb5BPqGeB5H6BTjzIplav',
 '0Z8c7bgYXsHvKDJ2eLUBXV',
 '2eC5qZtIGv427GEvbhkGei',
 '6Lmw9XlDKK5qvVQ2qk0mDk',
 '51wUFdgpNsV8cVzu7i6N0l',
 '4m1jesCFX1gQtIRFirfwCW',
 '02PT20DulS4wVbekDl4zhk',
 '4fQkGF3euv3UIoOXEXsPZo',
 '4zFPUEMucYleIIUnYVoeZf',
 '6kyqvHEWqcSlWObg2bQj6M',
 '07nm40KyxxOt3zCCTmewUR',
 '0JTg7E0zK9T2DgRbzv3OfY',
 '1oQLLvCmCSTiU9LoPstLlC',
 '4oI6nz5540UmFqDh4i3yay',
 '3nFtEpm4PZYPhV9G6nwDhK',
 '7MtVPRGtZl6rPjMfLoI3Lh',
 '55DyBUkjebkcYhDGfEXitR',
 '7erD8TKGNjHhNDqKAkEhMI',
 '6yb8ktQyg2mH4fOUeIYkd5',
 '2Jpw6uRsDncpCYkvdrfTTk',
 '69VfaraLe5onolFj90bgyj',
 '3pGgwcALM9UVOGb4y7JDio',
 

In [92]:
# pllist = spot_session.user_playlist_create(user=current_user_info['id'], name='test', public=True)
pllist

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/2Pk3OLPiVXIZHieWs5ZGHp'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/2Pk3OLPiVXIZHieWs5ZGHp',
 'id': '2Pk3OLPiVXIZHieWs5ZGHp',
 'images': [],
 'name': 'test',
 'owner': {'display_name': 'Agustinvargas',
  'external_urls': {'spotify': 'https://open.spotify.com/user/37t3cvb5u3o97hin4bsj40abw'},
  'href': 'https://api.spotify.com/v1/users/37t3cvb5u3o97hin4bsj40abw',
  'id': '37t3cvb5u3o97hin4bsj40abw',
  'type': 'user',
  'uri': 'spotify:user:37t3cvb5u3o97hin4bsj40abw'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MSwyZTU5NzQ1OWNlNWY4NGJhNTJkNDJiMDI1ZGJmNDBlOTY0MDMzOGJi',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/2Pk3OLPiVXIZHieWs5ZGHp/tracks',
  'items': [],
  'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0},
 'type': 'playlist',
 'uri': 'spotify:playlist:2Pk3OLPiVXIZHieWs5ZG

In [95]:
spot_session.user_playlist_add_tracks(user=current_user_info['id'], playlist_id=pllist['id'], tracks=trx_for_ply)

{'snapshot_id': 'Myw2ZDdiNzNiYmExZWE2MDNkMjE2NTk3NzQ0YjI0ZWQwZDgyYTQxOTMz'}

In [5]:
session2 = user.add_tracks(top_50_trx_ids=top_50_trx_ids, top_50_aud_feat=top_50_aud_feat, session=session)

In [14]:
# Create session
session = user.Session()

### First we create/find a user

### Instantiate token add the user1 data

In [16]:
# Instantiating User() object corresponding to user table in DB

user1 = User(spot_id='avargas-274', display_name='avargas-274')

### Instantiate a track

In [7]:
print('importing a track into the database for a given user and play list')
# trk2 = Tracks(
#             id=user_dict['Top Track IDs'][1],
#             danceability=user_dict['Track Audio Features'][1]['danceability'],
#             energy=user_dict['Track Audio Features'][1]['energy'],
#             key=user_dict['Track Audio Features'][1]['key'],
#             loudness=user_dict['Track Audio Features'][1]['loudness'],
#             mode=user_dict['Track Audio Features'][1]['mode'],
#             speechiness=user_dict['Track Audio Features'][1]['speechiness'],
#             acousticness=user_dict['Track Audio Features'][1]['acousticness'],
#             instrumentalness=user_dict['Track Audio Features'][1]['instrumentalness'],
#             valence=user_dict['Track Audio Features'][1]['valence'],
#             liveness=user_dict['Track Audio Features'][1]['liveness'],
#             tempo=user_dict['Track Audio Features'][1]['tempo'],
#             duration_ms=user_dict['Track Audio Features'][1]['duration_ms'],
#             time_signature=user_dict['Track Audio Features'][1]['time_signature']
#             )

importing a track into the database for a given user and play list


### Checking to see if song exists in DB if not then add it to db

In [27]:
trk_sesh = session.query(Tracks).filter(Tracks.id == trk2.id).first()
if trk_sesh is None:
    print(f"track with id {trk2.id} does not exists, so we'll up date it")
    session.add(trk2)
    session.commit()
else:
    print(f"The song with id {trk2.id} is already in our DB")

track with id 02gaYAEdeR6poHcBH1KUQF does not exists, so we'll up date it


### Creating `Tracks` objects, and adding to DB using `session.add_all()`

# Completed Functions for the `UserData` class
    Functions completed:
        - UserData().check_for_user(): checks if user exists in DB if
        - UserData().add_tracks(): checks to see if tracks exist in DB if they don't it updates them

In [30]:
def check_for_user(user_id, spot_cc):
    """
    Checks for user in DB.

    Input:
        - current_user_info: This object contains ID and displa_name for
          the user
        - spot_cc: Object for the spotify credentials connection

    Output:
        - : User() object from the my_db module
             contains q.id (user id) and q.display_name (user display name)
    """
    session = user.Session()
    engine = user.engine
    Base = user.base
    user_id = user_id
    spot_cc = spot_cc

    user1 = User(
                spot_id=user_id,
                )

    # update user and commit, then update token, and commit
    display_name_q = session.query(
                                    User
                                    ).filter(
                                            User.display_name ==
                                            user1.display_name
                                            ).first()

    # checking if user exists, if not then add user and token
    # if exists, check if token exists, if not add
    # if token exists, update using refresh_key
    if display_name_q is None:
        session.add(user1)
        session.commit()
        token = Tokens(
                    # Instead of "user_dict['Tokens Info']"" it should
                    # be "token_info"
                    access_token=token_info['access_token'],
                    token_type=token_info['token_type'],
                    expires_in=token_info['expires_in'],
                    refresh_token=token_info['refresh_token'],
                    scope=token_info['scope'],
                    expires_at=token_info['expires_at'],
                    # Still user "user1" because the id is not changing,
                    # simply adding mans to the db
                    user_token=user1
                    )
        session.add(token)
        session.commit()
    # Else, check if user has corresponding token that is not expired
    else:
        print(f"User {user1.display_name} already exists in DataBase;\
                checking for token")
        token_q = session.query(
                                Tokens
                                ).filter(
                                        Tokens.user == user1.id
                                        ).first()
        if token_q is None:
            print(f"We don't have a token for {user1.display_name},\
                    adding new token")
            token = Tokens(
                        # "token_info" to add tokens to db
                        access_token=token_info['access_token'],
                        token_type=token_info['token_type'],
                        expires_in=token_info['expires_in'],
                        refresh_token=token_info['refresh_token'],
                        scope=token_info['scope'],
                        expires_at=token_info['expires_at'],
                        # Still user "user1" because the id is not changing
                        # simply adding mans to the db
                        user_token=user1
                        )
            session.add(token)
            session.commit()
        # if the token exists refresh it, and update the value in table
        else:
            print(f"{user1.display_name} has a token, updating it")
            accs_token_refresh = spot_cc.refresh_access_token(
                                                    token_q.refresh_token
                                                    )
            token_info = spot_cc.get_access_token(as_dict=True)
            update_token = update(Tokens).where(
                                            Tokens.user == user1.id
                                                ).values(
                                                access_token=token_info[
                                                            'access_token'
                                                                ],
                                                expires_in=token_info[
                                                            'expires_in'
                                                                ],
                                                refresh_token=token_info[
                                                            'refresh_token'
                                                                ],
                                                expires_at=token_info[
                                                            'expires_at'
                                                                ]
                                                        )
            session.execute(update_token)
            session.commit()

    return session, user1

In [3]:
engine = create_engine(getenv('DATABASE_URL'))
Session = sessionmaker(autocommit=False, autoflush=False, bind=engine)
session = Session()
base = declarative_base()

user_id = 'avargas-274'

user_id_q = session.query(User).filter(User.spot_id == user_id).first()
user_id_q.display_name

'avargas-274'

In [6]:
def add_tracks(self, top_50_trx_ids, top_50_aud_feat, session):
    """
    This function is used when the user does not have their top 50 trx in
    our db, and we need to add them after adding the user.

    Input:
        - top_50_trx_ids: List of 50 alpha-neumeric track IDs
        - top_50_aud_feat: List of dicts containing feat data

    Output:
        - Updates DB with the track id and track feature data
        - DB session: object
    """
    top_50_aud_feat = top_50_aud_feat
    top_50_trx_ids = top_50_trx_ids
    session = session

    trx = []
    for k, v in enumerate(top_50_aud_feat):
        trk_sesh = session.query(
                                Tracks
                                ).filter(
                                        Tracks.id == top_50_trx_ids[k]
                                        ).first()
        if trk_sesh is not None:
            continue
        globals()['trk_' + str(k)] = Tracks(
                                    id=top_50_trx_ids[k],
                                    danceability=v['danceability'],
                                    energy=v['energy'],
                                    key=v['key'],
                                    loudness=v['loudness'],
                                    mode=v['mode'],
                                    speechiness=v['speechiness'],
                                    acousticness=v['acousticness'],
                                    instrumentalness=v['instrumentalness'],
                                    valence=v['valence'],
                                    liveness=v['liveness'],
                                    tempo=v['tempo'],
                                    duration_ms=v['duration_ms'],
                                    time_signature=v['time_signature']
                                    )
        trx.append(globals()['trk_' + str(k)])

    session.add_all(trx)
    session.commit()

    return session